## **Transformaciones MVP**


+ Generar campo **`id`**: Cada id se compondrá de la primera letra del nombre de la plataforma, seguido del show_id ya presente en los datasets (ejemplo para títulos de Amazon = **`as123`**)

+ Los valores nulos del campo rating deberán reemplazarse por el string “**`G`**” (corresponde al maturity rating: “general for all audiences”

+ De haber fechas, deberán tener el formato **`AAAA-mm-dd`**

+ Los campos de texto deberán estar en **minúsculas**, sin excepciones

+ El campo ***duration*** debe convertirse en dos campos: **`duration_int`** y **`duration_type`**. El primero será un integer y el segundo un string indicando la unidad de medición de duración: min (minutos) o season (temporadas)

## **Instalar Librerias**

In [11]:
#Transformaciones MVP
import polars as pl
import pandas as pd
import numpy as np

##Importar Dataframes

In [12]:
amazon_df = pd.read_csv('amazon_prime_titles.csv')
disney_plus_df = pd.read_csv('disney_plus_titles.csv')
netflix_title_df = pd.read_csv('netflix_titles.csv')
hulu_df = pd.read_csv('hulu_titles.csv')

## **Todos Los Def**

In [13]:
def CreandoListaColumna(dataframe,tituloDF):

#Se crea una lista y una variable que solo tiene una letra
#una vez por cada elemento en el dataframe, se añade un valor a lista que tiene la letra y el "show_id" propio de cada elemento
#todo eso se inserta en un dataframe, que es una copia pero ya tiene la lista como
  Lista=[]
  tituloDF = tituloDF[0]

  for e in range(dataframe.shape[0]):
    dato:str = tituloDF + (dataframe['show_id'][e])
    Lista.append(dato)
  dataframe.insert(0, 'id', Lista)
  return dataframe


def format_dates(df):   
    #Se cambia formato de datetime y se aplica al df original
    df['date_added'] = pd.to_datetime(df['date_added']).dt.strftime('%Y-%m-%d')
    return df


def lower_case(df):
    #Todo a minusculas
    string_columns = [column for column in df.columns if df[column].dtype == 'object']
    df[string_columns] = df.loc[:,string_columns].apply(lambda x: x.str.lower())
    return df


def duration(df):
  df[['duration_int', 'duration_type']] = df['duration'].str.split(expand=True)
  #Expand separa por el espacio
  df['duration_int'] = df['duration_int'].astype(float).astype('Int64')
  df.drop(columns='duration', inplace=True)
  return df



In [14]:
def Procesamiento(dataframe,tituloDF):
#Replace NaN for G
  dataframe['rating'].fillna(value='G', inplace=True)

#ID Modificado
  dataframe = CreandoListaColumna(dataframe,"tituloDF")

#Se cambia formato de datetime y se aplica al df original
  dataframe = format_dates(dataframe)

#Se pone todo en minusculas
  dataframe = lower_case(dataframe)

#Se Separa en dos columnas
  dataframe = duration(dataframe)

  return dataframe

## Ejecutando

In [15]:
amazon_df = Procesamiento(amazon_df,"Amason")
disney_plus_df = Procesamiento(disney_plus_df,"Disni")
netflix_title_df = Procesamiento(netflix_title_df,"Nepfi")
hulu_df = Procesamiento(hulu_df,"Hola")

In [67]:
amazon_df.head(1)

,id,show_id,type,title,director,cast,country,date_added,release_year,rating,listed_in,description,duration_int,duration_type
0,ts1,s1,movie,the grand seduction,don mckellar,"brendan gleeson, taylor kitsch, gordon pinsent",canada,2021-03-30,2014,g,"comedy, drama",a small fishing village must procure a local d...,113,min


In [66]:
#Guardarlos en su propia carpeta bien bonitos y lindos

amazon_df.to_csv('df_limpios/DF_Amazon.csv', index=False)
disney_plus_df.to_csv('df_limpios/DF_Disney.csv', index=False)
netflix_title_df.to_csv('df_limpios/DF_Netflix.csv', index=False)
hulu_df.to_csv('df_limpios/DF_Hulu.csv', index=False)

In [134]:
#Estoy 100% seguro de que hay una forma mucho mas rapida de hacer esto
#y de que se podria hacer sin unirlo, pero no quiero abrir
#8 csv por cada consulta

ruta_archivo1 = r"ratings/1.csv"
df1 = pd.read_csv(ruta_archivo1)
ruta_archivo2 = r"ratings/2.csv"
df2 = pd.read_csv(ruta_archivo2)
ruta_archivo3 = r"ratings/3.csv"
df3 = pd.read_csv(ruta_archivo3)
ruta_archivo4 = r"ratings/4.csv"
df4 = pd.read_csv(ruta_archivo4)
ruta_archivo5 = r"ratings/3.csv"
df5 = pd.read_csv(ruta_archivo5)
ruta_archivo6 = r"ratings/6.csv"
df6 = pd.read_csv(ruta_archivo6)
ruta_archivo7 = r"ratings/7.csv"
df7 = pd.read_csv(ruta_archivo7)
ruta_archivo8 = r"ratings/8.csv"
df8 = pd.read_csv(ruta_archivo8)

In [170]:
todojuntito = pd.concat([df1, df2, df3,df3,df4,df5,df6,df7,df8], axis=0)

In [171]:
todojuntito['timestamp'] = pd.to_datetime(todojuntito['timestamp'], unit='s')
todojuntito['year'] = todojuntito['timestamp'].dt.year
todojuntito.drop('timestamp', axis=1, inplace=True)

In [173]:
todojuntote = todojuntito.groupby(['movieId', 'year'])['rating'].mean().reset_index()


In [181]:
ordenado = todojuntote.sort_values('rating',ascending=False)
ordenado.head(2)

,movieId,year,rating
323053,ns1446,1998,5.0
136169,as6571,1998,5.0


In [184]:
ordenado["rating"] = ordenado["rating"].round(2)

In [185]:
ordenado.to_csv('ratings/GroupBy_Year_MovieId_MeanforRating.csv', index=False)

In [121]:
#jajaja re largos eran los promedios
promedios["rating"] = promedios["rating"].round(2)

In [122]:
promedios.to_csv('ratings/completo_agrupado_y_promediado.csv', index=False)


In [190]:
promedios.head(5)

,movieId,rating
0,as1,4.0
1,as10,3.5
2,as100,3.5
3,as1000,3.5
4,as1001,4.0


In [189]:
ordenadito = promedios.sort_values(by='rating', ascending=False)
ordenadito.head(2)

,movieId,rating
0,as1,4.0
18935,ns5269,4.0


In [187]:
#deberia haber cambiado esto con una funcion, pero vivan las expreciones regulares

# (^\w)(\w.*$)  -> $1,$2
# (^a) -> amazon
# (^n) -> netflix

#te amo expresiones regulares


In [191]:
ruta_completo = r"ratings/GroupBy_Year_MovieId_MeanforRating.csv"
completo_df = pd.read_csv(ruta_completo)

In [195]:
#4 archivos
df_rating_amazon = completo_df[completo_df['compania'] == 'amazon']
df_rating_netflix = completo_df[completo_df['compania'] == 'netflix']
df_rating_hulu = completo_df[completo_df['compania'] == 'hulu']
df_rating_disney = completo_df[completo_df['compania'] == 'disney']


In [234]:
#diccionario
#se crea una copia, se elimina año y score,
#se crea una tercer columna, solo para usarse y luego borrarse
#y se guarda el catalogo
completo_copia = completo_df.copy()
completo_copia['uniondeambos'] = completo_copia['compania'] + completo_copia['movieId']
completo_copia = completo_copia.drop_duplicates(subset=['uniondeambos'], keep='first')
completo_copia = completo_copia.drop(['uniondeambos'], axis=1)
completo_copia.to_csv('catalogo/catalogo_completo.csv', index=False)

In [221]:
amazon_df = pd.read_csv('df_limpios\DF_Amazon.csv',usecols=["show_id","type","title","country","duration_int","duration_type"])
disney_df = pd.read_csv('df_limpios\DF_Disney.csv',usecols=["show_id","type","title","country","duration_int","duration_type"])
hulu_df = pd.read_csv('df_limpios\DF_Hulu.csv',usecols=["show_id","type","title","country","duration_int","duration_type"])
netflix_df = pd.read_csv('df_limpios\DF_Netflix.csv',usecols=["show_id","type","title","country","duration_int","duration_type"])

In [270]:
netflix_df.head(1)

,show_id,type,title,release_year,duration_int,duration_type
0,s1,movie,dick johnson is dead,2020,90.0,min


In [272]:
max_duration = netflix_df['duration_int'].idxmax()
valor = netflix_df.iloc[max_duration]
print(valor)

show_id                               s4254
type                                  movie
title            black mirror: bandersnatch
release_year                           2018
duration_int                          312.0
duration_type                           min
Name: 4253, dtype: object


In [308]:
amazon_df.iloc[933].to_dict()

{'show_id': 's934',
 'type': 'movie',
 'title': 'soothing surf at del norte for sleep black screen',
 'release_year': 2018,
 'duration_int': 601,
 'duration_type': 'min'}

In [ ]:
'''Película (sólo película, no serie, etc) con mayor duración según año,
plataforma y tipo de duración.
La función debe llamarse get_max_duration(year, platform, duration_type)
y debe devolver sólo el string del nombre de la película.'''

In [269]:
amazon_df_max_duration = pd.read_csv('df_limpios\DF_Amazon.csv',usecols=["show_id","type","title","release_year","duration_int","duration_type"])
disney_df_max_duration = pd.read_csv('df_limpios\DF_Disney.csv',usecols=["show_id","type","title","release_year","duration_int","duration_type"])
hulu_df_max_duration = pd.read_csv('df_limpios\DF_Hulu.csv',usecols=["show_id","type","title","release_year","duration_int","duration_type"])
netflix_df_max_duration = pd.read_csv('df_limpios\DF_Netflix.csv',usecols=["show_id","type","release_year","title","duration_int","duration_type"])

In [268]:
amazon_df.head(2)

,show_id,type,title,release_year,duration_int,duration_type
0,s1,movie,the grand seduction,2014,113,min
1,s2,movie,take care good night,2018,110,min


In [ ]:
max_duration=ver_duraciones(str(plataforma),year)

In [309]:
score_movies_df = pd.read_csv('ratings\GroupBy_Year_MovieId_MeanforRating.csv')

In [310]:
score_movies_df.head(1)

,compania,movieId,year,rating
0,netflix,s1446,1998,5.0


In [333]:
def score_count(dataframe, scored, year):
#muestra la cantidad de peliculas con un score especifico
   compania = None

   if dataframe == "1" or dataframe == "amazon":
      compania = "amazon"
   elif dataframe == "2" or dataframe == "netflix":
      compania = "netflix"
   elif dataframe == "3" or dataframe == "disney":
      compania = "disney"
   elif dataframe == "4" or dataframe == "hulu":
      compania = "hulu"
   else:
      return "error"

   count_peliculas = score_movies_df.loc[(score_movies_df['compania'] == compania) & (score_movies_df['year'] == int(year)) & (score_movies_df['rating'] >= scored)] 
   return count_pelicula.shape[0]

In [334]:
cantidad=score_count("1", 3, "2015")
cantidad

NameError: name 'escored' is not defined